<a href="https://colab.research.google.com/github/KendallScott/Capstone/blob/main/Data%20API/New_Variables_TN_WV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.2 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=ab42d8dc6236016b4f047d502958648fe51e028f4ae793ec44bdf0af32573349
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=0700cfd718817a023511141a9c4cbfa4e54f70e052dee1287082db58db49a165
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
import eeconvert as eec

In [5]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
import pandas as pd
import os

directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['year'] = directory_files[z]
    df_file['year'] = df_file['year'].str.slice(3, 7)
    df_file['end_date'] = df_file['year']+'-07-31'
    df_file['start_date'] = df_file['year']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


In [19]:
df = df[df['State']=='wv']

In [20]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=59YW6ZgLbJuyjxl5WrAh5jZcGEkbFLcY8nQljmtT5VU&tc=82EN0uoP2H0LaRAY3sm1PIrn66e9iEmuoUpcRvh91DQ&cc=LGgpPAZZECKGtQuhF7zKAN5OsyrfWb5rKBKHhcYlOHE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmfTkU4XFcJk71kw-KJ6rnXHWnoL_cmlPhYQsa-OxhaA8HkeOApE4U

Successfully saved authorization token.


In [22]:
dataquery = df[['start_date','end_date','year', 'orig_perm_id', 'Company', 'mine_id']].drop_duplicates()
dataquery['end_date'] = dataquery['end_date'].astype(str)
dataquery['start_date'] = dataquery['start_date'].astype(str)

df_lat_long = df[['orig_perm_id', 'start_date','end_date', 'longitude', 'latitude', 'Company', 'mine_id']].groupby(['orig_perm_id', 'end_date', 'Company', 'mine_id']).last()
df_lat_long['start_date'] = df_lat_long['start_date'].astype(str)
dataquery = pd.merge(dataquery, df_lat_long,  how='inner', left_on=['orig_perm_id', 'Company', 'mine_id','start_date'], right_on = ['orig_perm_id', 'Company', 'mine_id','start_date'])

In [23]:
dataquery.year.unique()

array(['2023', '2021', '2019', '2017'], dtype=object)

In [ ]:
mines = []
temperature_metrics = []
water_quality_metrics = []
rain_metrics = []
aerosol_metrics = []
srad_metrics = []
vs_metrics = []
bi_metrics = []
fm1000_metrics = []
rmax_metrics = []

i=0
while i < len(dataquery):
    # Access the geometry of the feature
    data = dataquery[i:(i+1)]
    longitude = data['longitude'].values[0]
    latitude = data['latitude'].values[0]
    start_date = data['start_date'].values[0]
    end_date = data['end_date'].values[0]
    # Create an Earth Engine point geometry
    point = ee.Geometry.Point(longitude, latitude)

#https://medium.com/eelab/how-to-rank-the-most-polluted-water-body-in-city-using-sentinel-2-satellite-imagery-via-google-792685105bfd
#Write a function to calculate NDCI for every water pixel
#https://medium.com/@melqkiades/water-detection-using-ndwi-on-google-earth-engine-2919a9bf1951
    # Retrieve water quality metrics using satellite imagery
    water_quality = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first() \
        .normalizedDifference(['B3', 'B8']) \
        .rename('water_quality')

    temperature = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")\
        .filterBounds(point) \
        .filterDate(start_date, end_date) \
        .first() \

    rain = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('pr')
        #Precipitation amount

    temp = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('tmmx')
        #Maximum temperature

    rmax = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('rmax')

    srad = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('srad')

    vs = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('vs')

    bi = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('bi')

    fm1000 = temperature.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point,
        scale=30
        ).get('fm1000')
    # Calculate water quality statistics within a region of interest (e.g., buffer around the point)
    water_quality_stats = water_quality.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=point.buffer(500),  # Adjust buffer size as needed
        scale=10
    ).get('water_quality')


#https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_NRTI_L3_AER_AI#bands
#A measure of the prevalence of aerosols in the atmosphere.

    print(i)
    # Append the water quality metric to the list
    temperature_metrics.append(temp.getInfo())
    rain_metrics.append(rain.getInfo())

    srad_metrics.append(srad.getInfo())
    vs_metrics.append(vs.getInfo())
    bi_metrics.append(bi.getInfo())
    fm1000_metrics.append(fm1000.getInfo())
    rmax_metrics.append(rmax.getInfo())
    mines.append(data.mine_id)
    i+=1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# Add latitude, longitude, elevation, and water quality columns
data = pd.DataFrame()
data['Max Temperature'] = temperature_metrics
data['Precipitation'] = rain_metrics
data['srad'] = srad_metrics
data['vs'] = vs_metrics
data['bi'] = bi_metrics
data['fm1000'] = fm1000_metrics
data['rmax'] = rmax_metrics
data['mine_id'] = mines

# Print the updated DataFrame
print(data)

In [ ]:
file_name = 'wv_newvariables'
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)